### Imports

In [ ]:
!gdown 128UP6X4kbWVjjOKt4vB9bqqPVeT16cwL -O "covid.csv"
!gdown 1yj5Pa_Zck6VNf1JgkdCuErUKl5FLuoAd -O "hatecrime.csv"
!gdown 1yigT-1eM5Ki-uJA4FGpnt5bQDM0PtlKr -O "15m_cleaned_tweets.csv"
!gdown 19WLK_YzFvPnaEko-WllwClS0ZMVRdjHk -O  "stringency.csv"

Downloading...
From: https://drive.google.com/uc?id=128UP6X4kbWVjjOKt4vB9bqqPVeT16cwL
To: /content/covid.csv
100% 5.10M/5.10M [00:00<00:00, 154MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yj5Pa_Zck6VNf1JgkdCuErUKl5FLuoAd
To: /content/hatecrime.csv
100% 54.6M/54.6M [00:01<00:00, 53.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yigT-1eM5Ki-uJA4FGpnt5bQDM0PtlKr
To: /content/15m_cleaned_tweets.csv
100% 86.5M/86.5M [00:01<00:00, 60.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=19WLK_YzFvPnaEko-WllwClS0ZMVRdjHk
To: /content/stringency.csv
100% 43.4k/43.4k [00:00<00:00, 23.6MB/s]


In [ ]:
import pandas as pd

### Covid

This dataset contains information for the United States COVID-19 Cases and Deaths by State over Time. It was extracted from the Centers for Disease Control and Prevention (CDC). 

The code below is performing a series of data cleaning and preprocessing steps on the dataframe named "covid". The first step was to convert the "submission_date" column to a datetime object, allowing for proper sorting and filtering by date. The dataframe was then sorted by date and state and only the columns "submission_date","state","new_case" were selected. The dataframe was then filtered to include only rows between the date range of "2020-01-01" and "2021-03-31", as this specific timeframe overlapped with the other datasets being used, making it possible to combine and analyze the datasets together. The dataframe was then pivoted by index "date" and columns "state". Missing values were filled with 0. The index of **the** dataframe was set as "date". After that, the number of states contained in the dataframe was tested; and as it included more than 50 values, any location that was not included in the 50 US states was dropped. Then the data was resampled to a monthly sum. Lastly, the dataframe was then melted by "date",var_name="state",value_name="covid_cases" . This allow for easy aggregation and filtering of the data by date and state. The last step was to check the shape of the dataframe to make sure we have the right amount of states and dates.




In [ ]:
covid = pd.read_csv("covid.csv")
covid.head()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,03/11/2021,KS,297229,241035.0,56194.0,0,0.0,4851,NaN,NaN,0,0.0,03/12/2021 03:20:13 PM,Agree,NaN
1,12/01/2021,ND,163565,135705.0,27860.0,589,220.0,1907,NaN,NaN,9,0.0,12/02/2021 02:35:20 PM,Agree,Not agree
2,01/02/2022,AS,11,NaN,NaN,0,0.0,0,NaN,NaN,0,0.0,01/03/2022 03:18:16 PM,NaN,NaN
3,11/22/2021,AL,841461,620483.0,220978.0,703,357.0,16377,12727.0,3650.0,7,3.0,11/22/2021 12:00:00 AM,Agree,Agree
4,05/30/2022,AK,251425,NaN,NaN,0,0.0,1252,NaN,NaN,0,0.0,05/31/2022 01:20:20 PM,NaN,NaN


In [ ]:
covid = covid[covid["new_case"]>=0]

In [ ]:
covid["submission_date"] = pd.to_datetime(covid["submission_date"])

In [ ]:
covid = covid.sort_values(by=["submission_date","state"])[["submission_date","state","new_case"]]

In [ ]:
covid = covid.reset_index().drop(columns=["index"])

In [ ]:
covid.rename(columns={"submission_date":"date"},inplace=True)

In [ ]:
covid = covid[(covid["date"]>='2020-01-01') & (covid["date"]<="2021-03-31")]

In [ ]:
covid = covid.pivot(index='date', columns='state')['new_case'].reset_index().rename_axis(None,axis=1).fillna(0)

In [ ]:
covid.set_index('date',inplace=True) 

In [ ]:
covid.shape

(435, 60)

In [ ]:
us_abbreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
for state in covid.columns:
  if state not in us_abbreviations:
    print(state)

AS
DC
FSM
GU
MP
NYC
PR
PW
RMI
VI


In [ ]:
covid.drop(columns=["AS",
"DC",
"FSM",
"GU",
"MP",
"NYC",
"PR",
"PW",
"RMI",
"VI"],inplace=True)

In [ ]:
covid.head()

,AK,AL,AR,AZ,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
date,,,,,,,,,,,,,,,,,,,,,
2020-01-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2020-01-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-26,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
covid = covid.resample("M").sum()

In [ ]:
covid.reset_index(inplace=True)

In [ ]:
covid = covid.melt(id_vars="date",var_name="state",value_name="covid_cases")

In [ ]:
covid.head()

,date,state,covid_cases
0,2020-01-31,AK,0.0
1,2020-02-29,AK,0.0
2,2020-03-31,AK,128.0
3,2020-04-30,AK,227.0
4,2020-05-31,AK,108.0


In [ ]:
covid.shape

(750, 3)

### Crime

This section regards the Anti-Asian hate crimes dataframe. The dataset was extracted from XXX. 

The first step was to drop a number of columns that were not relevant to the analysis. Then the dataframe was filtered to only include rows where the value in the column bias description was equal to Anti-Asian. The incident date column was then converted to a datetime object and the data was grouped by state abbreviations and incident date, with values being summed, and the values sorted by incident date. The dataframe was then pivoted by index incident date and columns state abbreviations, filtering the total individual victims column and filling any missing values with 0. The index was reset and filtered to include only rows between the same date range as the covid cases dataset. The data was then resampled to a monthly sum by state and any location that was not part of the 50 US states were dropped. The data was melted by date, var_name="state", and value_name="hate_crimes".



In [ ]:
crime = pd.read_csv("hatecrime.csv")
crime.head()

,incident_id,data_year,ori,pug_agency_name,pub_agency_unit,agency_type_name,state_abbr,state_name,division_name,region_name,...,offender_race,offender_ethnicity,victim_count,offense_name,total_individual_victims,location_name,bias_desc,victim_types,multiple_offense,multiple_bias
0,43,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,Not Specified,1,Aggravated Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S
1,44,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,Not Specified,2,Aggravated Assault;Destruction/Damage/Vandalis...,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,M,S
2,45,1991,AR0600300,North Little Rock,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,Not Specified,2,Aggravated Assault;Murder and Nonnegligent Man...,2.0,Residence/Home,Anti-White,Individual,M,S
3,46,1991,AR0600300,North Little Rock,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,Not Specified,1,Intimidation,1.0,Residence/Home,Anti-White,Individual,S,S
4,47,1991,AR0670000,Sevier,NaN,County,AR,Arkansas,West South Central,South,...,White,Not Specified,1,Intimidation,1.0,School/College,Anti-Black or African American,Individual,S,S


In [ ]:
crime.drop(columns=['incident_id', 'ori', 'pug_agency_name', 'pub_agency_unit',
       'agency_type_name', 'state_name', 'division_name',
       'region_name','population_group_code', 'population_group_description',
       'adult_victim_count', 'juvenile_victim_count',
       'adult_offender_count',
       'juvenile_offender_count', 'offender_ethnicity',
       'offense_name', 
       'location_name', 'victim_types', 'multiple_offense',
       'multiple_bias'],inplace=True)

In [ ]:
crime = crime[crime["bias_desc"]=="Anti-Asian"]

In [ ]:
crime.drop(columns=["total_offender_count","offender_race","bias_desc","data_year","victim_count"],inplace=True)

In [ ]:
crime["incident_date"] = pd.to_datetime(crime["incident_date"])

In [ ]:
crime = crime.groupby(["state_abbr","incident_date"]).sum().reset_index().sort_values(by=["incident_date"])

In [ ]:
crime = crime.pivot(index='incident_date', columns='state_abbr')['total_individual_victims']
crime.fillna(0,inplace=True)

In [ ]:
crime  = crime.reset_index().rename_axis(None,axis=1)

In [ ]:
crime = crime[crime["incident_date"]>='2020-01-01']
crime.head()

,incident_date,AK,AL,AR,AZ,CA,CO,CT,DC,DE,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
4459,2020-01-01,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4460,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4461,2020-01-03,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4462,2020-01-04,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4463,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
crime  = crime.reset_index().drop(columns=["index"])

In [ ]:
crime.head()

,incident_date,AK,AL,AR,AZ,CA,CO,CT,DC,DE,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,2020-01-01,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-03,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-04,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
crime.rename(columns={"incident_date":"date"},inplace=True)

In [ ]:
crime = crime[(crime["date"]>='2020-01-01') & (crime["date"]<="2021-03-31")]

In [ ]:
crime.set_index('date',inplace=True) 
crime_by_month = crime.resample('M').sum()

In [ ]:
crime_by_month.rename(columns={"NB":"NE"},inplace=True)

*Disclaimer:  in 1969, the Canadian postal administration requested that the abbreviation for the state of Nebraska, which was originally NB, be changed to NE to prevent confusion with the province of New Brunswick in Canada.*

In [ ]:
crime_by_month.drop(columns=["DC","FS","GM"],inplace=True)

In [ ]:
crime_by_month.reset_index(inplace=True)

In [ ]:
crime_final = crime_by_month.melt(id_vars="date",var_name="state",value_name="hate_crimes")

In [ ]:
crime_final.head()

,date,state,hate_crimes
0,2020-01-31,AK,0.0
1,2020-02-29,AK,0.0
2,2020-03-31,AK,0.0
3,2020-04-30,AK,0.0
4,2020-05-31,AK,0.0


In [ ]:
crime_final.shape

(750, 3)

In [ ]:
crime_final["hate_crimes"].value_counts()

0.0     583
1.0      80
2.0      43
3.0      13
4.0       9
5.0       8
6.0       3
7.0       3
11.0      2
10.0      2
18.0      1
16.0      1
8.0       1
15.0      1
Name: hate_crimes, dtype: int64

### Tweets

This section concerns the anti-asian hate tweets. This file was pre-hydrated - see colab - in order to recover the location. The original dataset extracted from the paper originally contained 200M tweets. This dataset was reduced to 15M to maximise processing. 

This dataframe was filtered to only include rows where the value in the column "BERT_label" was equal to 1, indicating that the tweet was hateful. This considerably reduced the dataset as only hateful tweets were retained and tweets with a BERT label of 0, indicating no hate, were disregarded. The data was then grouped by date and location and the hate tweets counts were reset to the index. The "created_at" column was then converted to a datetime object and the data was sorted by date and location. The columns were then renamed to "date" and "hate_tweets" respectively, since the BERT label 1 were counted to represent the number of hate tweets per states over time rather than an indication of a hate tweet. 

The next step was to filter the "user_location" to only contain stattes. To do this, A list of all US state abbreviations was created and used to go through each location in the "user_location" column. Next, if a state abbreviation was present in a location, it was added to a new list called "sts". Once all locations were checked, the "user_location" column was replaced with the "sts" list containing only the state abbreviations found and the column was renamed to "state". The dataframe was then grouped by "date" and "state" and the sum was reset to the index. The dataframe was then pivoted by index "date" and columns "state" and filtered. The dataframe was then melted by date, var_name="state", and value_name="hate_tweets".

In [ ]:
tweets = pd.read_csv("/content/15m_cleaned_tweets.csv")
tweets.head()

,Unnamed: 0,id,user_location,created_at,BERT_label
0,4,1326918184126074886,"Phoenix, AZ",Nov 2020,0
1,5,1321164103721885697,"Chicago, IL",Oct 2020,0
2,7,1280645004596252672,"San Antonio, TX",Jul 2020,0
3,16,1337951731519549442,"Orlando, FL",Dec 2020,0
4,18,1226174047169458181,"Perrysburg, OH",Feb 2020,0


In [ ]:
tweets.drop(columns=["Unnamed: 0"],inplace=True)

In [ ]:
tweets.drop(columns=["id"],inplace=True)

In [ ]:
tweets = tweets[tweets["BERT_label"]==1]

In [ ]:
tweets = tweets.groupby(["created_at","user_location"]).count().reset_index()

In [ ]:
tweets["created_at"] = pd.to_datetime(tweets["created_at"])

In [ ]:
tweets= tweets.sort_values(by=["created_at","user_location"])

In [ ]:
tweets.rename(columns={"created_at":"date","BERT_label":"hate_tweets"},inplace=True)

In [ ]:
tweets = tweets.reset_index().drop(columns=["index"])

In [ ]:
us_abbreviations = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI',
       'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN',
       'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH',
       'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA',
       'WI', 'WV', 'WY']

In [ ]:
# Create empty list to store state abbreviations found in user_location column
# Iterate through each location in user_location column
# Iterate through each state abbreviation
sts = []
for loc in tweets.user_location:
  for state in us_abbreviations:
    if state in loc:
      sts.append(state)
      break


In [ ]:
# Replace user_location column with sts list containing only state abbreviations
# Rename user_location column to state

tweets["user_location"]=sts
tweets = tweets.rename(columns={"user_location":"state"})

In [ ]:
tweets = tweets[tweets["state"]!="DC"]
tweets["state"].nunique()

50

In [ ]:
tweets = tweets.reset_index().drop(columns=["index"])

In [ ]:
tweets = tweets.groupby(["date","state"]).sum().reset_index()

In [ ]:
tweets = tweets.pivot(index='date', columns='state')['hate_tweets'].reset_index().rename_axis(None,axis=1).fillna(0)

In [ ]:
tweets = tweets.melt(id_vars="date",var_name="state",value_name="hate_tweets")

In [ ]:
tweets.head()

,date,state,hate_tweets
0,2020-01-01,AK,1.0
1,2020-02-01,AK,0.0
2,2020-03-01,AK,2.0
3,2020-04-01,AK,0.0
4,2020-05-01,AK,2.0


In [ ]:
tweets.shape

(750, 3)

### Stringency Index


In [ ]:
stringency  = pd.read_csv("stringency.csv")
stringency.head()

,Date,State_Code,StringencyIndex,GovernmentResponseIndex,Population,party
0,2020-02-29,AK,15.328276,14.754138,731158.0,Republican
1,2020-03-31,AK,47.939032,40.555484,731158.0,Republican
2,2020-04-30,AK,86.697333,75.704000,731158.0,Republican
3,2020-05-31,AK,69.355484,66.059032,731158.0,Republican
4,2020-06-30,AK,55.000667,57.609333,731158.0,Republican


In [ ]:
columns_to_keep = ['Date', 'State_Code', 'party', 'StringencyIndex', 'GovernmentResponseIndex', 'Population']


In [ ]:
stringency = stringency[columns_to_keep]

In [ ]:
# group by 'State_Code' and resample by month to calculate the mean value of the stringency Index for each month
stringency['Date'] = pd.to_datetime(stringency['Date'], format='%Y-%m-%d')
stringency.set_index('Date', inplace=True, drop=False)
stringency_monthly = stringency.groupby('State_Code').resample('M').mean() 

In [ ]:
stringency_monthly.reset_index(inplace=True)

In [ ]:
stringency_monthly.shape

(650, 5)

In [ ]:
stringency_monthly = stringency_monthly.merge(stringency[["State_Code","party"]],on=["State_Code"],how="left").drop_duplicates()

In [ ]:
stringency_monthly.head()

,State_Code,Date,StringencyIndex,GovernmentResponseIndex,Population,party
0,AK,2020-02-29,15.328276,14.754138,731158.0,Republican
13,AK,2020-03-31,47.939032,40.555484,731158.0,Republican
26,AK,2020-04-30,86.697333,75.704000,731158.0,Republican
39,AK,2020-05-31,69.355484,66.059032,731158.0,Republican
52,AK,2020-06-30,55.000667,57.609333,731158.0,Republican


In [ ]:
stringency_monthly.shape

(650, 6)

### Merging All Into One

We merged Hate Tweets and Hate Crime with the Covid Cases 

In [ ]:
final = covid.merge(crime_final,on=["date","state"])


In [ ]:
final["hate_tweets"] = tweets["hate_tweets"]

In [ ]:
final.head()

,date,state,covid_cases,hate_crimes,hate_tweets
0,2020-01-31,AK,0.0,0.0,1.0
1,2020-02-29,AK,0.0,0.0,0.0
2,2020-03-31,AK,128.0,0.0,2.0
3,2020-04-30,AK,227.0,0.0,0.0
4,2020-05-31,AK,108.0,0.0,2.0


Lasly, we merged all datasets into one while also accounting for state population. We also added the data for the Stringency Index - measure of how "strict" the federal preventive COVID-19 policies were at a given time. 

In [ ]:
final.rename(columns={"date":"Date","state":"State_Code"},inplace=True)

In [ ]:
stringency_final = stringency_monthly.merge(final,on=["Date","State_Code"],how="left")

In [ ]:
stringency_final.drop(columns=["GovernmentResponseIndex","party"],inplace=True)

In [ ]:
def func(col):
  pop = col.values[0]
  con = col.values[1]
  return con/pop*1000000

In [ ]:
stringency_final["covid_cases"] = stringency_final[["Population","covid_cases"]].apply(func,axis=1)
stringency_final["hate_tweets"] = stringency_final[["Population","hate_tweets"]].apply(func,axis=1)
stringency_final["hate_crimes"] = stringency_final[["Population","hate_crimes"]].apply(func,axis=1)

In [ ]:
stringency_final.drop(columns="Population",inplace=True)

In [ ]:
stringency_final.head()

,State_Code,Date,StringencyIndex,covid_cases,hate_crimes,hate_tweets
0,AK,2020-02-29,15.328276,0.000000,0.0,0.000000
1,AK,2020-03-31,47.939032,175.064760,0.0,2.735387
2,AK,2020-04-30,86.697333,310.466411,0.0,0.000000
3,AK,2020-05-31,69.355484,147.710891,0.0,2.735387
4,AK,2020-06-30,55.000667,657.860545,0.0,0.000000


In [ ]:
stringency_final.to_csv("stringency_final.csv",index=False)